In [1]:
# python
import sys
import os
import importlib
# columnar analysis
from coffea.nanoevents import NanoEventsFactory
import awkward as ak
# local
sys.path.insert(1, os.path.join(sys.path[0], '../..')) # fixme: there must be a better way to handle this...
from sidm.tools import ffschema, sidm_processor, utilities
# always reload local modules to pick up changes during development
importlib.reload(ffschema)
importlib.reload(sidm_processor)
importlib.reload(utilities)
# plotting
import matplotlib.pyplot as plt
utilities.set_plot_style()

In [2]:
sample_dir = "root://xcache//store/group/lpcmetx/SIDM/ffNtupleV4/2018/"
sample_name = "SIDM_XXTo2ATo2Mu2E_mXX-100_mA-1p2_ctau-9p6"
sample_file = f"{sample_dir}/{sample_name}_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18DRPremix-102X_upgrade2018_realistic_v15-v1/210326_161703/0000/ffNtuple_1.root"

sample_name = '4mu_500GeV_5GeV_0p08mm'
samples = [sample_name]
fileset = utilities.make_fileset(samples, "llpNanoAOD_v2")
# create events collection from single file
fname = fileset[samples[0]][0]

events = NanoEventsFactory.from_root(
    sample_file,
    treepath='ffNtuplizer/ffNtuple',
    schemaclass=ffschema.FFSchema,
    metadata = {
        "dataset": sample_name, 
    },
).events()

In [4]:
p = sidm_processor.SidmProcessor(
    [
        "4mu",
        "2mu2e",
        "base",
        "baseNoLj",
    ],
    ["base"],
    #-0.4 means use the ljsource collection to cluster jets with radius 0.4, 
    # 0 means use the lepton jets directly from the ntuples
    # Any positive number is the radius of the jet, clustered with the photon, electron, muon, and dsa muon collections. 
    # So 0.4 means cluster ljs from the individual objects with a distance parameter of 0.4
    lj_reco_choices=["0.4"]
)
out = p.process(events)

Not applying any cuts to the lepton jets for channel baseNoLj
Uh oh, haven't evaluated this cut yet! Make sure it was included in the list of cuts you used to initialize this JaggedSelection.   electrons :  looseID
Uh oh, haven't evaluated this cut yet! Make sure it was included in the list of cuts you used to initialize this JaggedSelection.   muons :  looseID
Uh oh, haven't evaluated this cut yet! Make sure it was included in the list of cuts you used to initialize this JaggedSelection.   photons :  looseID
Uh oh, haven't evaluated this cut yet! Make sure it was included in the list of cuts you used to initialize this JaggedSelection.   dsaMuons :  pT > 10 GeV
Uh oh, haven't evaluated this cut yet! Make sure it was included in the list of cuts you used to initialize this JaggedSelection.   dsaMuons :  |eta| < 2.4
Uh oh, haven't evaluated this cut yet! Make sure it was included in the list of cuts you used to initialize this JaggedSelection.   dsaMuons :  DT + CSC hits > 12
Uh oh, hav

ValueError: key "pt" does not exist (not in record)

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.3/src/libawkward/util.cpp#L525)

In [ ]:
out

In [ ]:
utilities.plot(out[sample_name]["hists"]["pv_n"]["0","base",:])

In [ ]:
out[sample_name]["cutflow"]["0"]["2mu2e"].print_table()
print()
print("Unweighted, ntuple ljs (Option 0):")
out[sample_name]["cutflow"]["0"]["2mu2e"].print_table(unweighted=True)
print()
print("Unweighted, ljs from lj_source (Option -0.4):")
out[sample_name]["cutflow"]["-0.4"]["2mu2e"].print_table(unweighted=True)
print()
print("Unweighted, ljs from individual collections, dR < 0.4:")
out[sample_name]["cutflow"]["0.4"]["2mu2e"].print_table(unweighted=True)
print()
print("Unweighted, ljs from individual collections, dR < 0.2:")
out[sample_name]["cutflow"]["0.2"]["2mu2e"].print_table(unweighted=True)

In [ ]:
out[sample_name]["cutflow"]["0"]["4mu"].print_table()
print()
print("Unweighted, ntuple ljs (Option 0):")
out[sample_name]["cutflow"]["0"]["4mu"].print_table(unweighted=True)
print()
print("Unweighted, ljs from lj_source (Option -0.4):")
out[sample_name]["cutflow"]["-0.4"]["4mu"].print_table(unweighted=True)
print()
print("Unweighted, ljs from individual collections, dR < 0.4:")
out[sample_name]["cutflow"]["0.4"]["4mu"].print_table(unweighted=True)
print()
print("Unweighted, ljs from individual collections, dR < 0.2:")
out[sample_name]["cutflow"]["0.2"]["4mu"].print_table(unweighted=True)

In [ ]:
out[sample_name]["cutflow"]["0"]["2mu2e"].print_table(fraction=True)
print()
out[sample_name]["cutflow"]["0"]["2mu2e"].print_table(fraction=True, unweighted=True)

In [ ]:
out[sample_name]["cutflow"]["0"]["4mu"].print_table()
out[sample_name]["cutflow"]["0"]["4mu"].print_table(unweighted=True)

In [ ]:
utilities.plot(out[sample_name]["hists"]["pv_ndof"]["0","2mu2e", :])

In [ ]:
utilities.plot(out[sample_name]["hists"]["lj_eta_phi"]["0","2mu2e", :, :])

In [ ]:
utilities.plot(out[sample_name]["hists"]["lj_pt"]["0","2mu2e", ::2j])

In [ ]:
utilities.plot([
    out[sample_name]["hists"]["mu_lj_pt"]["0","2mu2e", ::2j],
    out[sample_name]["hists"]["egm_lj_pt"]["0","2mu2e", ::2j],
])
plt.legend(["Muon-type LJ", "EGamma-type LJ"])

In [ ]:
ljs = events.ljsource
#ljs = ak.mask(ljs, ak.num(ljs) >= 2)
ljs = ljs[ak.argsort(ljs.pt, ascending=False)]
ljs.pt[ak.num(ljs) >= 2, 1]
print(ak.num(ljs.pt))
print(ljs.pt)
print(events.weightProduct)

In [ ]:
reco_choices = ["-0.4", "0", "0.2", "0.4"]
utilities.plot([out[sample_name]["hists"]["lj0_pt"][lj, "2mu2e", :] for lj in reco_choices])
plt.legend(reco_choices)
# somehow 

In [ ]:
utilities.plot([out[sample_name]["hists"]["lj1_pt"][lj, "2mu2e", :] for lj in reco_choices])
plt.legend(reco_choices)

In [ ]:
utilities.plot([out[sample_name]["hists"]["lj_lj_invmass"][lj, "2mu2e", :300j] for lj in reco_choices])
plt.legend(reco_choices)

In [ ]:
utilities.plot([out[sample_name]["hists"]["lj_lj_absdphi"][lj, "2mu2e", :300j] for lj in reco_choices])
plt.legend(reco_choices)

In [ ]:
out[sample_name]["hists"]["lj_lj_absdphi"].axes[0]

In [ ]:
d = {
    "pv": events["pv"],
}
a = {
    "pv": d["pv"][:1800, :],
}
a["pv"]